# Prueba A/B para una tienda internacional.

En esta prueba A/B para una tienda donde analizaremos los resultados relacionados con la introduccion de un sistema de recomendaciones mejorado con un grupo de control (A) y  un grupo de prueba que usará el nuevo embudo de pago (B). Tenemos previsto 6000 participantes. 

Lo que esperamos de esta prueba A/B es que dentro de los 14 días posteriores a la inscripción, los usuarios mejoren estas métricas o eventos:

- Una mejor conversión en vistas de la página del producto (el evento product_page).
- Instancias de agregar artículos al carrito de compras (product_cart)
- Compras (purchase)
- Al menos un 10% de mejora en cada etapa de este embudo (product_page → product_cart → purchase).

# Descripción de los sets

Contaremos con 4 sets de datos que contienen estas columnas:
    
1. ab_project__marketing_events_us.csv:

* name — el nombre del evento de marketing
* regions — regiones donde se llevará a cabo la campaña publicitaria
* start_dt — fecha de inicio de la campaña
* finish_dt — fecha de finalización de la campaña

2. final_ab_new_users_upd_us.csv:

* user_id
* first_date — fecha de inscripción
* region
* device — dispositivo utilizado para la inscripción

3. final_ab_events_upd_us.csv:

* user_id
* event_dt — fecha y hora del evento
* event_name — nombre del tipo de evento
* details — datos adicionales sobre el evento (por ejemplo, el pedido total en USD para los eventos purchase)

4. final_ab_participants_upd_us.csv:

* user_id
* ab_test — nombre de la prueba
* group — el grupo de prueba al que pertenecía el usuario

In [1]:
# Importar librerías y sets de datos necesarios

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as mth
from scipy import stats as st
import scipy.stats as stats

In [3]:
first_events_data = pd.read_csv('./datasets/ab_project_marketing_events_us.csv')
first_events_data

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


In [4]:
users_data = pd.read_csv('./datasets/final_ab_new_users_upd_us.csv')
users_data

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
...,...,...,...,...
58698,1DB53B933257165D,2020-12-20,EU,Android
58699,538643EB4527ED03,2020-12-20,EU,Mac
58700,7ADEE837D5D8CBBD,2020-12-20,EU,PC
58701,1C7D23927835213F,2020-12-20,EU,iPhone


In [5]:
events_data = pd.read_csv('./datasets/final_ab_events_upd_us.csv')
events_data

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99
...,...,...,...,...
423756,245E85F65C358E08,2020-12-30 19:35:55,login,NaN
423757,9385A108F5A0A7A7,2020-12-30 10:54:15,login,NaN
423758,DB650B7559AC6EAC,2020-12-30 10:59:09,login,NaN
423759,F80C9BDDEA02E53C,2020-12-30 09:53:39,login,NaN


In [6]:
test_data = pd.read_csv('./datasets/final_ab_participants_upd_us.csv')
test_data

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,4FF2998A348C484F,A,recommender_system_test
...,...,...,...
14520,1D302F8688B91781,B,interface_eu_test
14521,3DE51B726983B657,A,interface_eu_test
14522,F501F79D332BE86C,A,interface_eu_test
14523,63FBE257B05F2245,A,interface_eu_test


# Exploración inicial de los datos

**Exploración y cambios para el set de datos first_events_data**

In [7]:
first_events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


En este caso vemos que las columnas start_dt y finish_dt pueden ser cambiadas a formato datetime64 para un mejor manejo.

In [8]:
#haciendo el cambio de ambas columans a formato datetime
first_events_data['start_dt'] = pd.to_datetime(first_events_data['start_dt'])
first_events_data['finish_dt'] = pd.to_datetime(first_events_data['finish_dt'])
first_events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 580.0+ bytes


In [9]:
first_events_data.duplicated().sum()

0

**Exploración y cambios para el set de datos users_data**

In [10]:
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


Encontramos que el set de datos users_data tiene la columna first_date que se puede cambiar a formato datetime para un manejo más sencillo.

In [11]:
#cambiando columna first_date a formato datetime
users_data['first_date'] = pd.to_datetime(users_data['first_date'])
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     58703 non-null  object        
 1   first_date  58703 non-null  datetime64[ns]
 2   region      58703 non-null  object        
 3   device      58703 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.8+ MB


In [12]:
users_data.duplicated().sum()

0

**Exploración y cambios para el set de datos events_data**

In [13]:
events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB


La columna event_dt puede ser cambiada a tipo date_time para simplificar el analisis.

In [14]:
#haciendo el cambio de event_dt a tipo datetime
events_data['event_dt'] = pd.to_datetime(events_data['event_dt'])
events_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     423761 non-null  object        
 1   event_dt    423761 non-null  datetime64[ns]
 2   event_name  423761 non-null  object        
 3   details     60314 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 12.9+ MB


In [15]:
events_data.duplicated().sum()

0

**Exploración y cambios para el set de datos test_data**

In [16]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB


In [17]:
test_data.duplicated().sum()

0

En este último set de datos no necesitamos hacer ningún cambio.

Hemos hecho cambios a formato datetime en las columnas que necesitaban este cambio, y no hemos encontrado ningún duplicado en ningún set de datos y únicamente se encontraron valores nulos en la columna details del set de datos events_data que contiene detalles adicionales sobre el evento.

# Analisis exploratorio de datos

In [22]:
#creando df con los usaurios que se encuentran en ambos grupos

#users_to_remove = test_data[test_data['ab_test'].isin(['recommender_system_test', 'interface_eu_test'])].groupby('user_id').filter(lambda x: len(x) > 1)['user_id'].unique()

# Eliminamos los usuarios del dataframe original
test_data = test_data[~test_data['user_id'].isin(users_to_remove)]

# Mostramos el dataframe resultante
print(test_data)

                user_id group                  ab_test
0      D1ABA3E2887B6A73     A  recommender_system_test
1      A7A3664BD6242119     A  recommender_system_test
4      4FF2998A348C484F     A  recommender_system_test
5      7473E0943673C09E     A  recommender_system_test
6      C46FE336D240A054     A  recommender_system_test
...                 ...   ...                      ...
14520  1D302F8688B91781     B        interface_eu_test
14521  3DE51B726983B657     A        interface_eu_test
14522  F501F79D332BE86C     A        interface_eu_test
14523  63FBE257B05F2245     A        interface_eu_test
14524  79F9ABFB029CF724     B        interface_eu_test

[12751 rows x 3 columns]


In [23]:
new_test_data = test_data.query('ab_test == "recommender_system_test"')
new_test_data

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
4,4FF2998A348C484F,A,recommender_system_test
5,7473E0943673C09E,A,recommender_system_test
6,C46FE336D240A054,A,recommender_system_test
...,...,...,...
3670,053FB26D6D49EDDC,A,recommender_system_test
3671,9D263B8EF15CF188,B,recommender_system_test
3672,F2FBBA33F37DEC46,A,recommender_system_test
3673,29C92313A98B1176,B,recommender_system_test


In [24]:
new_test_data = new_test_data.merge(events_data, on='user_id', how='left')
new_test_data

,user_id,group,ab_test,event_dt,event_name,details
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,purchase,99.99
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:29,product_cart,NaN
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,product_page,NaN
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,login,NaN
4,A7A3664BD6242119,A,recommender_system_test,2020-12-20 15:46:06,product_page,NaN
...,...,...,...,...,...,...
18189,6715343AFBA285AE,B,recommender_system_test,2020-12-07 10:12:15,login,NaN
18190,6715343AFBA285AE,B,recommender_system_test,2020-12-08 22:51:16,login,NaN
18191,6715343AFBA285AE,B,recommender_system_test,2020-12-09 02:28:03,login,NaN
18192,6715343AFBA285AE,B,recommender_system_test,2020-12-10 22:55:14,login,NaN


In [25]:
new_test_data = new_test_data.merge(users_data, on='user_id', how='left')
new_test_data

,user_id,group,ab_test,event_dt,event_name,details,first_date,region,device
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,purchase,99.99,2020-12-07,EU,PC
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:29,product_cart,NaN,2020-12-07,EU,PC
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,product_page,NaN,2020-12-07,EU,PC
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07 14:43:27,login,NaN,2020-12-07,EU,PC
4,A7A3664BD6242119,A,recommender_system_test,2020-12-20 15:46:06,product_page,NaN,2020-12-20,EU,iPhone
...,...,...,...,...,...,...,...,...,...
18189,6715343AFBA285AE,B,recommender_system_test,2020-12-07 10:12:15,login,NaN,2020-12-07,CIS,Android
18190,6715343AFBA285AE,B,recommender_system_test,2020-12-08 22:51:16,login,NaN,2020-12-07,CIS,Android
18191,6715343AFBA285AE,B,recommender_system_test,2020-12-09 02:28:03,login,NaN,2020-12-07,CIS,Android
18192,6715343AFBA285AE,B,recommender_system_test,2020-12-10 22:55:14,login,NaN,2020-12-07,CIS,Android


In [26]:
new_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18194 entries, 0 to 18193
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     18194 non-null  object        
 1   group       18194 non-null  object        
 2   ab_test     18194 non-null  object        
 3   event_dt    18194 non-null  datetime64[ns]
 4   event_name  18194 non-null  object        
 5   details     2390 non-null   float64       
 6   first_date  18194 non-null  datetime64[ns]
 7   region      18194 non-null  object        
 8   device      18194 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 1.2+ MB


In [ ]:
print(new_test_data.dtypes)

In [ ]:
# Calculamos la diferencia en días directamente
new_test_data['days_between_events'] = (new_test_data['event_dt'] - new_test_data['first_date']).dt.days

# Filtramos los datos según las condiciones
new_test_data = new_test_data.query('days_between_events <= 14 & first_date <= "2021-01-01" & region == "EU"')

# Mostramos el resultado
new_test_data

# Funnel v2 

In [27]:
new_test_data.groupby('event_name')['event_name'].count().sort_values(ascending=False)

event_name
login           8233
product_page    5114
product_cart    2457
purchase        2390
Name: event_name, dtype: int64

# **Empezamos a crear el funnel** 

In [ ]:
events = new_test_data.pivot_table(index='user_id', columns='event_name', values='event_dt', aggfunc='min')
events.head()

In [ ]:
step_1 = ~events['login'].isna()
step_2 = step_1 & (events['product_page'] > events['login'])
step_3 = step_2 & (events['product_cart'] > events['product_page'])
step_4 = step_3 & (events['purchase'] > events['product_cart'])

n_login = events[step_1].shape[0]
n_product_page = events[step_2].shape[0]
n_product_cart = events[step_3].shape[0]
n_purchase = events[step_4].shape[0]

print('Entrantes:', n_login)
print('Aparición de la pantalla del producto:', n_product_page)
print('Aparicion de la pantalla del carrito de compras:', n_product_cart)
print('Aparicion de la pantalla del pago:', n_purchase)

In [ ]:
1 / 2593

In [ ]:
Únicamente 0,03% de los usaurios realizó el recorrido desde el evento 1. 
Exploiremos otras formas en las que se hicieron compras.

In [ ]:
step_1 = ~events['product_page'].isna()
step_2 = step_1 & (events['product_cart'] > events['product_page'])
step_3 = step_2 & (events['purchase'] > events['product_cart'])

n_product_page = events[step_1].shape[0]
n_product_cart = events[step_2].shape[0]
n_purchase = events[step_3].shape[0]

print('Aparición de la pantalla del producto:', n_product_page)
print('Aparicion de la pantalla del carrito de compras:', n_product_cart)
print('Aparicion de la pantalla del pago:', n_purchase)

In [ ]:
1/1632



Se puede observar que de los 1632 usuarios que empezaron el recorrido desde otro evento, solo 13 terminaron el pedido, lo cual equivale al 0.06% de usuarios. No muchos completaron el recorrido

In [ ]:
step_1 = ~events['product_cart'].isna()
step_2 = step_1 & (events['purchase'] > events['product_cart'])

n_product_cart = events[step_1].shape[0]
n_purchase = events[step_2].shape[0]

print('Aparicion de la pantalla del carrito de compras:', n_product_cart)
print('Aparicion de la pantalla del pago:', n_purchase)

In [ ]:
3/773


Se puede observar que de los 773 usuarios que empezaron el recorrido desde el penúltimo evento, solo 3 terminaron el pedido, lo cual equivale al 0.4% de usuarios. De la misma forma, No muchos completaron el recorrido.

Estudiar la conversión en las diferentes etapadas del embudo.

¿El número de eventos por usuario está distribuido equitativamente entre las muestras?

In [ ]:
new_test_data['group'].value_counts()

In [ ]:
new_test_data.shape[0]

 
En el registro de las pruebas hay 16959 eventos registrados, y el número en ambas muestras no esta distribuido equitativaente siendo la prueba A con el mayor número de eventos registrados, con una amplia diferencia de la prueba B.

¿Hay usuarios que estén presentes en ambas muestras?

In [ ]:
unique_A_user = new_test_data.query('group == "A"')['user_id'].unique()
unique_B_user = new_test_data.query('group == "B"')['user_id'].unique()
print(set(unique_B_user).issubset(set(unique_A_user)))

¿Cómo se distribuye el número de eventos entre los días?

In [ ]:
new_test_data['day'] = new_test_data['event_dt'].dt.weekday
new_test_data.head()

In [ ]:

new_test_data.hist(column="event_dt", bins=10)
plt.xticks(rotation=45)
plt.show()

In [ ]:
new_test_data['day'].value_counts().plot(kind='bar')


El día del 21 de diciembre del 2020 es donde más eventos estan distribuídos, asimismo, son los días domingos donde más ocurren estos eventos.

Ahora los datos se encuentran listos para realizar las pruebas A/B y poder comprobar los cambios relacionados con la introducción de un sistema de recomendaciones mejorado.

¿Hay alguna peculiaridad en los datos que hay que tener en cuenta antes de iniciar la prueba A/B?

# Evaluar los datos de la prueba A/B

In [ ]:
¿Qué podemos decir sobre los resultados de la prueba A/B?

In [ ]:
Prueba Z para comprobar la diferencia estadística de proporciones.

¿Qué puedes decir sobre los resultados de la prueba A/B según la diferencia estadítica de ambas proporciones? * 

In [ ]:
conversions = new_test_data[['user_id', 'group']].drop_duplicates()
converted = pd.DataFrame(data={
    "user_id": new_test_data[new_test_data["event_name"] == "purchase"]["user_id"].unique(),
    "converted": 1
})
conversions = conversions.merge(converted, on="user_id", how="left")
conversions["converted"] = conversions["converted"].fillna(0)
conversions.head()

In [ ]:
purchases = np.array([(conversions.query('group == "A" and converted == 1')['converted'].count()), 
                      conversions.query('group == "B" and converted == 1')['converted'].count()])
leads = np.array([(conversions.query('group == "A"')['group'].count()), 
                  (conversions.query('group == "B"')['group'].count())])

significance = 0.05
p1 = purchases[0]/leads[0]
p2 = purchases[1]/leads[1]
p_combined = ((purchases[0]+purchases[1])/(leads[0]+leads[1]))
difference = p1 - p2

z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/leads[0]+1/leads[1]))

distr = st.norm(0,1)

p_value = (1 - distr.cdf(abs(z_value))) * 2

print('p-value:', p_value)

if (p_value < significance):
    print('Rechazar la hipótesis nula: hay una diferencia significativa entre las proporciones.')
else:
    print('No se pudo rechazar la hipótesis nula: no hay razón para pensar que las proporciones son diferentes.')

Se puede observar que no hay una diferencia estadística en ambas proporciones; esto quiere decir que los datos fueron correctamente ordenados.



# Conclusión respecto a la etapa EDA y los resultados de la prueba

En este proyecto, se analizaron los 4 datsets, para observar los tipos, la cantidad de datos; y se realizaron distintas conversiones en algunos tipos de columnas al tipo correcto.

Asimismo, en el registro de los datos no se encontraron datos duplicados; además se encontraon valores ausentes, pero esto ocurrió debido a que estos valores ausentes estaban relacionados con una columna, es decir, dependían de los datos de dicha columna.

Para calcular la conversión de todos los eventos, se realizó un embudo de eventos para conocer el número de usuarios que pasan por cada etapa del evento, y llegan hasta la etapa final de los eventos; resultando en una conversión baja en cada etapa de los eventos. Asimismo, se encontró que es el día 21 de diciembre del 2020 donde más eventos ocurren.

Se pudo observar que hay usuarios en ambos grupos, y que hay diferencias entre ambos grupos, es decir, hay usuarios únicos en el A que no estan presentes en el B. Esto quiere decir que los datos no fueron divididos correctamente.

En este proyecto, no se ha especificado el nivel de significancia utilizado para probar las hipótesis estadísticas. Por lo tanto, se decidió establecer un nivel de significancia del 0.05.

Por último, se pudo observar que en las pruebas de los grupos no se encontraron diferencias significativas en las proporciones en cada uno de los eventos; por lo tanto se puede concluir que los datos se encuentran correctamente ordenados.

Como recomendación, se tendría que cenfocar más en el grupo de control A, ya que son los de mayor proporción y cantidad.

In [19]:
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

# Filtrar eventos relevantes
relevant_events = events_data[events_data['event_name'].isin(['visit_page', 'add_to_cart', 'purchase'])]

# Unir con el grupo del usuario
merged_data = relevant_events.merge(test_data, on='user_id')

# Agregar conversiones por grupo y evento
conversion_rates = (
    merged_data.groupby(['ab_test', 'group', 'event_name'])
    .agg(users=('user_id', 'nunique'))
    .reset_index()
)

# Calcular tasas de conversión
total_users = test_data.groupby(['ab_test', 'group'])['user_id'].nunique().reset_index()
conversion_rates = conversion_rates.merge(total_users, on=['ab_test', 'group'])
conversion_rates['conversion_rate'] = conversion_rates['users'] / conversion_rates['user_id']

# Comparar tasas con Z-test para cada evento
for event in ['visit_page', 'add_to_cart', 'purchase']:
    event_data = conversion_rates[conversion_rates['event_name'] == event]
    count = event_data['users'].values
    nobs = event_data['user_id'].values
    
    stat, pval = proportions_ztest(count, nobs)
    print(f"Evento: {event}, Estadístico: {stat}, p-valor: {pval}")

    

NotImplementedError: more than two samples are not implemented yet